In [ ]:
from basic_task import BasicTask
from corba import CorbaServer
import numpy as np

# Plan
#### 1. Explore basic setup to solve a task
#### 2. Look on functions more close

## 1.1 Create task
Basic task contacins Panda robot, table and one cuboid object

In [ ]:
# killall hppcorbaserver
corba_server = CorbaServer()
task = BasicTask()

## 1.2 Solve easy task
- set start and goal configurations (robot + object)
- clean any previous data from  problem solver
- find a path

In [ ]:
o1_p1 = [0.4, 0.4, 0.1]
q = [0, 0, 0, 1]
q_init = task.robot.initial_configuration() + o1_p1 + q
o1_p2 = [0.4, 0.1, 0.1]
q_goal = task.robot.initial_configuration() + o1_p2 + q
task.ps.setInitialConfig(q_init)
task.ps.addGoalConfig(q_goal)

task.ps.clearRoadmap()
for i in range(task.ps.numberPaths() - 1, -1, -1):
    task.ps.erasePath(i)
task.ps.addPathOptimizer("RandomShortcut")
task.ps.solve()


## 1.3 Visualize solution

In [ ]:
fps = 10
path_id = task.ps.numberPaths() - 1
nframes = np.ceil(task.ps.pathLength(path_id) * fps).astype(np.int)
configs = [task.ps.configAtParam(path_id, t) for t in np.linspace(0., task.ps.pathLength(path_id), nframes)]

task.render.visualise_configurations(configs)

## 2.1 Apply state constraints

State - **free**, object_i **grasped** by gripper_j, pregrasp, intersec, preplace.
In free all objects are subgect to placement constraint: objects are static on contact surfaces
In grasp one object (as we have only one gripper) is subject to grasp constraint: object is fixed w.r.t griper, other objects are subgect to placement constraint.

In [ ]:
q_init = task.robot.initial_configuration() + o1_p1 + q
succ0, q_init_constr, err0 = task.cg.graph.applyNodeConstraints(task.cg.nodes['free'], q_init)
configs = [q_init, q_init_constr]
task.render.visualise_configurations(configs)

In [ ]:
err0

## 2.2 Sample from transition
this function is used any time you want to sample a random config in a certain state

*q_from should lie in the "left" state of transition*

We will study transitions:

**Loop free** (`Loop | f`) - random config from free state, objects poses are static, diff robot configurations

**Loop grasp** (`'Loop | 0-0'`) - random config where gripper is grasping object using the first handle

**Pregrasp** (`'panda/gripper > cuboid/handleZpx | f_01'`) - first waypoint to grasp an object

**Intersec** (`'panda/gripper > cuboid/handleZpx | f_12'`) - second waypoint to grasp an object

**Preplace** (`'panda/gripper > cuboid/handleZpx | f_23'`) - third waypoint to grasp an object


In [ ]:
# task.cg.edges
n_try = 20
transitions = ['Loop | f', 
               'panda/gripper > cuboid/handleZpx | f_01',
               'panda/gripper > cuboid/handleZpx | f_12', 
#                'panda/gripper > cuboid/handleZpx | f_23', 
               'Loop | 0-0']
q_from = q_init.copy()
for trans in transitions:
    configs = []
    for i in range(n_try):
        succ, q1, err = task.cg.generateTargetConfig(trans, q_from, task.robot.shootRandomConfig())
        if not succ:
            continue
        q1_open_fingers = task.robot.modify_open_gripper(q1.copy())
        res, msg = task.robot.isConfigValid(q1_open_fingers)  # Collision checking
        if not res:
            continue
        configs.append(q1_open_fingers)
    task.render.visualise_configurations(configs)
    q_from = configs[-1].copy()
    print(f"In {trans} state {len(configs)} configs generated")

## 2.3 Direct path
direct path will succeed if configs are in the same state, or in neighboring state (but order maters)

In [ ]:
def sample_from_trans(q_from, trans, n_try=50):
    for i in range(n_try):
        succ, q1, err = task.cg.generateTargetConfig(trans, q_from, task.robot.shootRandomConfig())
        if not succ:
            continue
        q1_open_fingers = task.robot.modify_open_gripper(q1.copy())
        res, msg = task.robot.isConfigValid(q1_open_fingers)  # Collision checking
        if not res:
            continue
        return q1_open_fingers

In [ ]:
q_from = q_init.copy()
q_to = sample_from_trans(q_from, 'panda/gripper > cuboid/handleZpx | f_01')

In [ ]:
res, pid, msg = task.ps.directPath(q_from, q_to, True)
if not res:
    res, pid, msg = task.ps.directPath(q_to, q_from, True)
if res:
    fps = 1
    path_id = task.ps.numberPaths() - 1
    nframes = np.ceil(task.ps.pathLength(pid) * fps).astype(np.int)
    configs = [task.ps.configAtParam(pid, t) for t in np.linspace(0., task.ps.pathLength(pid), nframes)]

    task.render.visualise_configurations(configs)
else:
    print("direct path failed")